In [1]:
%%bash
pip install tifffile -q

In [2]:
import os
import gc
import cv2
import math
import random as r
import numpy as np
import pandas as pd
import tifffile as tifi

from time import sleep

sleep_time: float = 3.5

In [3]:
def breaker(num: int=50, char: str="*") -> None:
    print("\n" + num*char + "\n")


def get_ram_usage() -> None:
    print(f"RAM Usage : {psutil.virtual_memory().percent / 100 * 16:.2f} GB")
    

def preprocess_keep_aspect_ratio(image: np.ndarray, size: int=1024) -> np.ndarray:
    h, w, _ = image.shape

    scale = min(size / w, size / h)
    nh, nw = math.ceil(h * scale), math.ceil(w * scale)

    image = cv2.resize(src=image, dsize=(nw, nh), interpolation=cv2.INTER_AREA)
    new_image = np.ones((size, size, 3), dtype=np.uint8) * 127
    hh: int = (size - nh) // 2
    ww: int = (size - nw) // 2

    if abs(nh-(size - 2*hh)) == 1: new_image[hh:size-hh-1, ww:size-ww, :] = image
    elif abs(nw-(size - 2*ww)) == 1: new_image[hh:size-hh, ww:size-ww-1, :] = image
    else: new_image[hh:size-hh, ww:size-ww, :] = image
    
    return new_image


def preprocess(image: np.ndarray, size: int=1024) -> np.ndarray:
    return cv2.resize(src=image, dsize=(size, size), interpolation=cv2.INTER_AREA)


def show(image: np.ndarray, title: str=None) -> None:
    plt.figure()
    plt.imshow(image)
    if title: plt.title(title)
    plt.show()

In [4]:
df = pd.read_csv("../input/mayo-clinic-strip-ai/train.csv")
df.head(5)

,image_id,center_id,patient_id,image_num,label
0,006388_0,11,006388,0,CE
1,008e5c_0,11,008e5c,0,CE
2,00c058_0,11,00c058,0,LAA
3,01adc5_0,11,01adc5,0,LAA
4,026c97_0,4,026c97,0,CE


In [5]:
filenames = df.image_id.copy().values
# filenames = filenames[:10]

size: int = 1024

for filename in filenames:
    image = tifi.imread(f"../input/mayo-clinic-strip-ai/train/{filename}.tif")
    image = preprocess(image, size)
    cv2.imwrite(f"{filename}.png", image)
    
    del image
    gc.collect()
    sleep(sleep_time)